## René Parlange, MSc
### 📚 Machine Learning Course, PhD in Computer Science
#### 🎓 Instructor: Juan Carlos Cuevas Tello, PhD
#### 🏛 Universidad Autónoma de San Luis Potosí (UASLP)

🔗 [GitHub Repository](https://github.com/parlange/ml-notebooks)

# scikit-learn BernoulliRBM with Optuna hyperparameter search

## install optuna with pip and ! (terminal command) in Colab

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00


## dataset: MNIST

In [ ]:
import numpy as np
import optuna
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml, load_digits
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
import time

# Start time measurement
start_time = time.time()

digits = load_digits()
X = digits.data
Y = digits.target.astype(int)
X_scaled = minmax_scale(X, feature_range=(0, 1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

def objective(trial):
    n_components = trial.suggest_int("n_components", 50, 200, step=50)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)
    batch_size = trial.suggest_int("batch_size", 10, 50)
    n_iter = trial.suggest_int("n_iter", 10, 20, step=5)

    rbm = BernoulliRBM(n_components=n_components, learning_rate=learning_rate, batch_size=batch_size, n_iter=n_iter, random_state=42)
    classifier = LogisticRegression(max_iter=1000, solver="saga")
    pipeline = Pipeline(steps=[('rbm', rbm), ('classifier', classifier)])

    return cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(f"Best trial score: {study.best_value}")
print(f"Best trial params: {study.best_params}")

best_params = study.best_params
rbm_best = BernoulliRBM(n_components=best_params["n_components"], learning_rate=best_params["learning_rate"], batch_size=best_params["batch_size"], n_iter=best_params["n_iter"], random_state=42)
classifier_best = LogisticRegression(max_iter=1000, solver="saga")
pipeline_best = Pipeline(steps=[('rbm', rbm_best), ('classifier', classifier_best)])
pipeline_best.fit(X_train, y_train)

y_pred_best = pipeline_best.predict(X_test)
print('\nClassification Report with Best Model:\n', classification_report(y_test, y_pred_best, zero_division=1))

# End time measurement and print execution time
end_time = time.time()
print(f"\nExecution time: {end_time - start_time:.2f} seconds")

[I 2023-10-18 02:31:37,509] A new study created in memory with name: no-name-93836f17-0207-400c-b36f-4568adb118ee
[I 2023-10-18 02:31:41,120] Trial 0 finished with value: 0.6464857341684064 and parameters: {'n_components': 150, 'learning_rate': 0.018029010525609453, 'batch_size': 28, 'n_iter': 20}. Best is trial 0 with value: 0.6464857341684064.
[I 2023-10-18 02:31:42,291] Trial 1 finished with value: 0.49617258176757134 and parameters: {'n_components': 50, 'learning_rate': 0.013642758658827001, 'batch_size': 25, 'n_iter': 20}. Best is trial 0 with value: 0.6464857341684064.
[I 2023-10-18 02:31:47,320] Trial 2 finished with value: 0.7125956854558106 and parameters: {'n_components': 200, 'learning_rate': 0.01463050031361257, 'batch_size': 18, 'n_iter': 15}. Best is trial 2 with value: 0.7125956854558106.
[I 2023-10-18 02:31:48,318] Trial 3 finished with value: 0.6089074460681977 and parameters: {'n_components': 50, 'learning_rate': 0.03330768498402832, 'batch_size': 40, 'n_iter': 20}. B

Best trial score: 0.8935281837160751
Best trial params: {'n_components': 150, 'learning_rate': 0.09934471281294344, 'batch_size': 10, 'n_iter': 20}

Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        33
           1       0.89      0.89      0.89        28
           2       0.94      0.94      0.94        33
           3       1.00      0.85      0.92        34
           4       1.00      0.98      0.99        46
           5       0.93      0.83      0.88        47
           6       0.97      0.97      0.97        35
           7       0.94      0.97      0.96        34
           8       0.76      0.87      0.81        30
           9       0.78      0.88      0.82        40

    accuracy                           0.92       360
   macro avg       0.92      0.92      0.92       360
weighted avg       0.92      0.92      0.92       360


Execution time: 261.65 seconds
